# Study of numerical methods for ODEs on the Hénon-Heiles differential equation system
# Aruran Sivashankar

## Introduction

In this report I am going to implement several numerical methods for solving ODEs and apply them on the Hénon-Heiles differential equation system to observe their properties. First we're going to show the rate of convergence for each of the methods, which are 

- Shampine-Bogacki method of order 3
- Kutta's method of order 4
- Kahan's method of order 2
- Störmer-Verlet method of order 2

For each method we are then going to determine their ability to preserve energy, and plot their Poincaré-map. In this report we will use constant step size, and assume autonomous systems only. 

## General comments

I have tried to implement the numerical methods to handle a arbitrary system of differential equations, this however proved to be quite difficult for Kahan's method and Störmer-Verlet method. The implementations are still quite generally applicable, but there might be a need to modify some values in the functions. The implementations are mainly explained as text before the relevant numerical method. 

Hardware used for time measurement:
- CPU: Ryzen 7 5600x (not overclocked)
- RAM: DDR4 32GB 3600 Mhz

### Import useful libraries

In [ ]:
import numpy as np 
from scipy import integrate 
from matplotlib import pyplot as plt 
from numba import jit 
import time 

plt.rcParams['figure.figsize'] = [25, 10]

## Utility functions

The first function is for plotting. There are many commands that are used for every plot, and I have collected them into one function to make the code for readable. 

The second function is for calculating the number of times we have to call a method for a given step size and time interval. The step size will not necessarily be a multiple of the time interval, so the function calculates the step size for the last step, and every implementation calculates the last step seperately to account for the possible difference in the last step size. 

In [ ]:
def plotCommands(xlabel, ylabel, title, xscale = False, yscale = False, legend = False, printString = ""):
    #Function containing the usual commands when plotting with matplotlib.pyplot
    #If log-scaled x- and/or y-axis is needed, set xscale and/or yscale to True
    #If the figure has a label, set legend = True
    #If extra information is needed, use printString
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    if xscale:
        plt.xscale('log')
    if yscale:
        plt.yscale('log')
    if legend:
        plt.legend()
    plt.grid()
    print('\033[1m' + "Figure ", plotCommands.counter)#Bold letters
    if printString:
        print(printString)
    plt.show()
    plotCommands.counter += 1
    
    return None

def plotCommandsSubPlots(xlabel, ylabel, title):#Function when plt.subplot is used
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid()

    return None

@jit(nopython = True)
def create_H(t_0, t_end, h):
    #Calculate number of iterations to compute, with a given h
    #if remainder is not zero the last step is the remainder
    #Otherwise the last step is h
    ratio = (t_end - t_0)/h
    remainder = (t_end - t_0)%h
    n = int(ratio) + 1 #Add one to include the intial condition
    if (remainder):
        return (n + 1), remainder#Add 1 to n to include the last step
    else:
        return n, h

In [ ]:
plotCommands.counter = 1 # To keep track of figure numbers

## Implementation of Hénon-Heiles system

First we are going to define the Hénon-Heiles system for each of the methods. The reason for the specific implementation choices will be explained below, but in short terms there are 4 different implementations. They are all designed to take in values by passing-by-reference instead of calculating and returning the values. This way we save both memory space and execution time. The only exception is the system for the library method, which I couldn't make work with that method, and is therefore quite slow compared to the implemented methods. It is also slow because the scipy-library will not allow the use of "@jit(nopython = True)" function decorator from the numba library, which compiles a given function.

The Hénon-Heiles system is a system of dimension 4 and the equations are:

$$
q_1' = p_1\\
q_2' = p_2\\
p_1' = -q_1(1+2q_2)\\
p_2' = -(q_2 + q_1^2 - q_2^2)
$$

The shape of the y-matricies containing the solutions for the function will always be on the form:

$$
\vec{y} = \left[\vec{p_1} \quad \vec{p_2} \quad \vec{q_1} \quad \vec{q_2}\right]^T
$$

in the entire project. 

### Hénon-Heiles system for the scipy library method "RK45"

In [ ]:
@jit(nopython = True)
def henon_heiles_library(t, y):#Hénon-Heiles system for the scipy library method 'RK45'
    #The scipy-library numerical method requires both time and values as arugment, even if our equation are not explicitly time dependant
    dp1 = -y[2]*(1 + 2*y[3])
    dp2 = -(y[3] + y[2]**2 - y[3]**2)
    dq1 = y[0]
    dq2 = y[1]
    
    return [dp1, dp2, dq1, dq2]

### Hénon-Heiles system for Shapmine-Bogacki and Kutta's method

In [ ]:
@jit(nopython = True)
def henon_heiles(y, Y_vec):#Hénon-Heiles system for Shapmine-Bogacki and Kutta's method
    y2 = y[2]
    y3 = y[3]
    Y_vec[0] = -y2*(1 + 2*y3)
    Y_vec[1] = -(y3 + y2**2 - y3**2)
    Y_vec[2] = y[0]
    Y_vec[3] = y[1]

### Hénon-Heiles system in matrix form for Kahan's method

In [ ]:
@jit(nopython = True)
def create_A(y, h, A):
    #Given an identity matrix fill in the rest of the values of A
    A[0][2] = h*(y[3] + 0.5)
    A[0][3] = h*y[2]
    A[1][2] = h*y[2]
    A[1][3] = h*(0.5 - y[3])
    A[2][0] = -0.5*h
    A[3][1] = -0.5*h

@jit(nopython = True)
def create_b(y, h, b):
    #Given an array with the same length as y, fill in for b
    b[0] = y[0] - 0.5*h*y[2]
    b[1] = y[1] - 0.5*h*y[3]
    b[2] = y[2] + 0.5*h*y[0]
    b[3] = y[3] + 0.5*h*y[1]

### Hénon-Heiles system adapted for Störmer-Verlet numerical method

In [ ]:
@jit(nopython = True)
def henon_heiles_SV(y, h, Y_vec, q_next):#Henon_heiles for Störmer-Verlet method
    p1_half = y[0] + q_next[0] #0.5*h*(-y[2]*(1 + 2*y[3]))
    q1_next = y[2] + h * p1_half
    p2_half = y[1] + q_next[1] #0.5*h*(-(y[3] + y[2]**2 - y[3]**2))
    q2_next = y[3] + h * p2_half

    #This way we only need to calculate the function once every step besides the first
    #The calculations that wouldv'e happened every time are commented beside the corresponding line of code
    q_next[0] =  0.5 * h * (-q1_next*(1 + 2*q2_next))
    q_next[1] = 0.5 * h * (-q2_next - q1_next**2 + q2_next**2)

    Y_vec[0] = p1_half + q_next[0] #0.5 * h * (-q1_next*(1 + 2*q2_next))
    Y_vec[1] = p2_half + q_next[1] #0.5 * h * (-q2_next + q1_next**2 - q2_next**2)
    Y_vec[2] = q1_next
    Y_vec[3] = q2_next 


@jit(nopython = True)
def henon_heiles_SV_last(y, last_step, Y_vec, q_next):#Implementation of Störmer-Verlet for the last step
    q_next[0] =  0.5 * last_step * (-y[2]*(1 + 2*y[3]))#We need the q-values to be calculated with the last step size
    q_next[1] = 0.5 * last_step * (-y[3] - y[2]**2 + y[3]**2)

    p1_half = y[0] + q_next[0] 
    q1_next = y[2] + last_step * p1_half
    p2_half = y[1] + q_next[1] 
    q2_next = y[3] + last_step * p2_half

    q_next[0] =  0.5 * last_step * (-q1_next*(1 + 2*q2_next))
    q_next[1] = 0.5 * last_step * (-q2_next - q1_next**2 + q2_next**2)

    Y_vec[0] = p1_half + q_next[0]
    Y_vec[1] = p2_half + q_next[1] 
    Y_vec[2] = q1_next
    Y_vec[3] = q2_next 

## Implementation of numerical methods

The first method I am going to implement is the Shampine-Bogacki method. The Shampine-Bogacki method is a Runge-Kutta method of order 3. The method is defined as:

$$
F_1 = F(y_n)\\
F_2 = F(y_n + \frac{1}{2} h F_1)\\
F_3 = F(y_n + \frac{3}{4} h F_2)\\
y_{n+1} = y_n + h\left(\frac{2}{9} F_1 + \frac{1}{3} F_2 + \frac{4}{9} F_3\right)
$$

for a differential equation on the form $$y' = F(y)$$

Because we are going to deal with a problem of dimension 4 we can consider y a vector:

$$\vec{y}' = F(\vec{y})$$

### Shampine-Bogacki method of order 3

In [ ]:
#Shampine-Bogacki method
@jit(nopython = True) 
def shampine_bogacki(t_0, t_end, y_0, h, f):
    n, last_step = create_H(t_0, t_end, h)# Get number of iterations and the size of the last step in case (t_end - t_0)%h != 0

    T = np.zeros(n) #Array containg the time for each step
    T[0] = t_0

    Y = np.zeros((len(y_0), n)) #initialize a matrix to be of the same dimension as the initial-condition
    Y[:,0] = y_0
    Y_vec = np.zeros((3, len(y_0)))#Our method requires 3 stored values for each step
    
    #Calculate with constant h up to next to last step
    for i in range(n - 2):
        #Pick out the correct y-values
        y = Y[:,i]

        #Calculate with the Shampine-Bogacki method
        f(y, Y_vec[0])
        f(y + 0.5*h*Y_vec[0], Y_vec[1])
        f(y + 0.75*h*Y_vec[1], Y_vec[2])

        #Append correct values to the Y-matrix and time-array
        Y[:,i + 1] = y + h/9*(2 * Y_vec[0] + 3 * Y_vec[1] + 4 * Y_vec[2])
        T[i + 1] = T[i] + h

    #Use last_step as step size to calculate the last step
    y = Y[:,-2]
    f(y, Y_vec[0])
    f(y + 0.5*last_step*Y_vec[0], Y_vec[1])
    f(y + 0.75*last_step*Y_vec[1], Y_vec[2])

    Y[:,-1] = y + last_step/9 * (2 * Y_vec[0] + 3 * Y_vec[1] + 4 * Y_vec[2])
    T[-1] = T[-2] + last_step

    return T, Y        

The second method is Kutta's method, which is a Runge-Kutta method of order 4, and is defined as: 

$$
F_1 = F(y_n)\\
F_2 = F(y_n + \frac{1}{2} h F_1)\\
F_3 = F(y_n + \frac{1}{2} h F_2)\\
F_4 = F(y_n + h F_3)\\
y_{n+1} = y_n + h\left(\frac{1}{6} F_1 + \frac{1}{3} F_2 + \frac{1}{3} F_3+\frac{1}{6} F_4\right)
$$

for equation on the form:

$$
\vec{y} = F(\vec{y})
$$

The implementation for Shampine-Bogacki and Kutta's method  is straight-forward, as we use the previous substep to evaluate the next, and this happens 3 and 4 times each step respectively. For efficiency I have chosen to pass arguments by reference to the function as explained in the "implementation of Hénon-Heiles" subsection.

#### Kutta's method of order 4

In [ ]:
#Kutta's method
@jit(nopython = True)
def kuttas_method(t_0, t_end, y_0, h, f):
    n, last_step = create_H(t_0, t_end, h)

    T = np.zeros(n) #Array containg the time for each step
    T[0] = t_0

    Y = np.zeros((len(y_0), n)) #initialize a matrix to be of the same dimension as the initial-condition
    Y[:,0] = y_0
    Y_vec = np.zeros((4, len(y_0)))#Our method requires four stored values for each step

    #Calculate with constant h up to next to last step
    for i in range(n - 2):
        #Pick out the correct y-values
        y = Y[:,i]

        #Calculate with the Shampine-Bogacki method
        f(y, Y_vec[0])
        f(y + 0.5*h*Y_vec[0], Y_vec[1])
        f(y + 0.5*h*Y_vec[1], Y_vec[2])
        f(y + h*Y_vec[2], Y_vec[3])

        #Append correct values to the Y-matrix and time-array
        Y[:,i + 1] = y + h/6*(Y_vec[0] + 2*Y_vec[1] + 2*Y_vec[2] + Y_vec[3])
        T[i + 1] = T[i] + h
    
    #Use last_step as step size to calculate the last step
    y = Y[:,-2]
    f(y, Y_vec[0])
    f(y + 0.5*last_step*Y_vec[0], Y_vec[1])
    f(y + 0.5*last_step*Y_vec[1], Y_vec[2])
    f(y + last_step*Y_vec[2], Y_vec[3])

    Y[:,-1] = y + last_step/6*(Y_vec[0] + 2*Y_vec[1] + 2*Y_vec[2] + Y_vec[3])
    T[-1] = T[-2] + last_step

    return T, Y


#Kutta's method with scipy library, cannot use jit
def kuttas_method_library(t_0, t_end, y_0, f, a_tol, r_tol):
    solution = integrate.solve_ivp(f, (t_0, t_end), y_0, method = 'RK45', atol = a_tol, rtol = r_tol, vectorized = True)
    return solution.t, solution.y

The third method is Kahan's method which is designed for a quadratic system of differential equations. A quadratic component can be written on the form:

$$
y_i' = \sum_{j,k = 1}^{m} a_{ijk} y_j y_k + \sum_{j=1}^{m}b_{ij} + c_i, \quad i = 1,...,m
$$

Kahan's method is then defined as:

$$
\frac{y_{i,n+1} - y_{i, n}}{h} = \sum_{j,k = 1}^{m} a_{ijk} \frac{y_{j,n} y_{k, n+1} + y_{j,n+1} y_{k, n}}{2} + \sum_{j = 1}^{m} b_{ij} \frac{y_{j,n} + y_{j, n+1}}{2} + c_i
$$

First we define the following variables:

$$
y_0 = p_1, \quad y_1 = p_1, \quad y_2 = q_1, \quad y_3 = q_2
$$

and we get that the Hénon-Heiles system is:

$$
y_2' = y_0\\
y_3' = y_1\\
y_0' = -y_2(1+2y_3)\\
y_1' = -(y_3 + y_2^2 - y_3^2)
$$

By using the definition of Kahan's method we get the following equations:

$$
y_{0,n+1} + h\left(y_{3,n} + \frac{1}{2}\right)y_{2,n+1} + h y_{2,n} y_{3,n+1} = y_{0,n} - \frac{h}{2}y_{2,n}\\
y_{1,n+1} + h y_{2,n} y_{2,n+1} + h\left(\frac{1}{2} - y_{3,n}\right)y_{3,n+1}  = y_{0,n} - \frac{h}{2}y_{2,n}\\
y_{2, n+1} - \frac{h}{2}y_{0,n+1} = y_{2,n} + \frac{h}{2} y_{0,n}\\
y_{3, n+1} - \frac{h}{2}y_{1,n+1} = y_{3,n} + \frac{h}{2} y_{1,n}
$$

Here I have collected all the terms with $n+1$ on the same side, and we can rewrite this system of equations as the following matrices:

$$
A = 
\begin{pmatrix} 
1 & 0 &  h\left(y_{3,n} + \frac{1}{2}\right) & h y_{2,n}\\ 
0 & 1 & h y_{2,n} & h\left(\frac{1}{2} - y_{3,n}\right)\\
-\frac{h}{2} & 0 & 1 & 0\\
0 & -\frac{h}{2} & 0 & 1\\
\end{pmatrix}
\quad\quad
b = 
\begin{pmatrix}
y_{0,n} - \frac{h}{2}y_{2,n}\\
y_{1,n} - \frac{h}{2}y_{3,n}\\
y_{2,n} + \frac{h}{2}y_{0,n}\\
y_{3,n} + \frac{h}{2}y_{1,n}\\
\end{pmatrix}
\quad\quad
x = 
\begin{pmatrix}
y_{0, n+1}\\
y_{1, n+1}\\
y_{2, n+1}\\
y_{3, n+1}\\
\end{pmatrix}
$$

And solve $Ax = b$ for x for each step. This is accomplished with the numpy-library function numpy.linalg.solve(A,b), which will return x. The matricies are created by calling the functions create_A and create_b, defined above. 

#### Kahans method of order 2

In [ ]:
@jit(nopython = True)
def kahans_method(t_0, t_end, y_0, h, create_A, create_b):
    #Input are the functions that create matricies A and b
    n, last_step = create_H(t_0, t_end, h)
    m = len(y_0)

    Y = np.zeros((m, n))
    Y[:,0] = y_0

    T = np.zeros(n)
    T[0] = t_0

    A = np.identity(m)
    b = np.zeros(m)

    #Calculate with constant h up to next to last step
    for i in range(n - 2):
        #Create the matrix A, and array b
        create_A(Y[:,i], h, A)
        create_b(Y[:,i], h, b)
        Y[:,i+1] = np.linalg.solve(A, b)#solve and append to Y matrix containing the solutions
        T[i+1] = T[i] + h

    #Use last_step as step size to calculate the last step
    create_A(Y[:,-2], last_step, A)
    create_b(Y[:,-2], last_step, b)
    Y[:,-1] = np.linalg.solve(A, b)
    T[-1] = T[-2] + last_step

    return T, Y

Finally we have the Störmer-Verlet method which is designed for second order differential equation on the form $q'' = f(q)$, which can be rewritten as a first order system with dimension 2 by defining $p := q'$, and we get:

$$
q' = p \quad\quad \text{and} \quad\quad p' = f(q)
$$

We can then define the Störmer-Verlet method as:

$$
p_{n + \frac{1}{2}} = p_n + \frac{1}{2} h f(q_n)\\
q_{n + 1} = q_n +  h p_{n + \frac{1}{2}}\\
p_{n + 1} = p_{n + \frac{1}{2}} + \frac{1}{2} h f(q_{n + 1})\\
$$

Because the Hénon-Heiles system is composed of two such systems, and they depend on each other, we have to solve them in parallel. The implementation of the Hénon-Heiles system is therefore:

$$
p_{1, n + \frac{1}{2}} = p_{1, n} + \frac{1}{2} h f(q_{1,n},q_{2,n})\\
p_{2, n + \frac{1}{2}} = p_{2, n} + \frac{1}{2} h f(q_{1,n},q_{2,n})\\

q_{1, n + 1} = q_{1, n} +  h p_{1, n + \frac{1}{2}}\\
q_{2, n + 1} = q_{2, n} +  h p_{1, n + \frac{1}{2}}\\


p_{1, n + 1} = p_{1, n + \frac{1}{2}} + \frac{1}{2} h f(q_{1, n + 1}, q_{2, n + 1})\\
p_{2, n + 1} = p_{2, n + \frac{1}{2}} + \frac{1}{2} h f(q_{1, n + 1}, q_{2, n + 1})\\
$$

For efficiency the first step only the first and last step requires two evaluations of the functions. All other steps only require 1, the last requires 2 evaluations because we potentially have a different step size for as the last step. 

#### Störmer-Verlet method of order 2

In [ ]:
@jit(nopython = True)
def stormer_verlet(t_0, t_end, y_0, h, f, f_last):
    n, last_step = create_H(t_0, t_end, h)#Get number of steps and last step size

    T = np.zeros(n)#Time
    T[0] = t_0

    Y = np.zeros((len(y_0), n)) #initialize a matrix to be of the same dimension as the initial-condition
    Y[:,0] = y_0
    q_next = np.array([0.5*h*(-y_0[2]*(1 + 2*y_0[3])), 0.5*h*(-y_0[3] - y_0[2]**2 + y_0[3]**2)]) #First evaluation for the first step

    #Calculate with constant h up to next to last step
    for i in range(n - 2):
        f(Y[:,i], h, Y[:,i+1], q_next)
        T[i + 1] = T[i] + h

    #Use last_step as step size to calculate the last step, with the correct function
    f_last(Y[:,-2], last_step, Y[:,-1], q_next)
    T[-1] = T[-2] + last_step

    return T, Y

## Problem 1 - Test of rate of convergence

For the test of rate of convergence for each of the methods, I have chosen to compare the methods against the library-method 'RK45' from the scipy-library. This method uses the Dormand-Prince pair formulas, and is a Runge-Kutta method of order 5 ([reference[2]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.RK45.html)). For a small time interval I will solve the Hénon-Heiles system with all the numerical methods implemented. The step size will vary logarithmically from $10^{-3.9}$ to $1$. At each step size the last values for each numerical method will be compared to the library method, which will use a relative tolerance of approximately $2.22 \cdot 10^{-14}$, which is the lowest possible tolerance allowed by the library. The difference between the two methods will determine our error, which we will see might be problematic for smaller step sizes. However this works excellently for relatively small step sizes, and for methods with lower order of convergence. As we will observe the only method with a problem for these values is Kutta's method, because of its high accuracy. 

The errors for each method is plotted on a loglog-plot, with a reference line with the order of method as the slope. The reference line is on the form $$k \cdot H ^ {\text{order of convergence}}$$. The value of k will be the right-most value for the error, so the lines line up in the plot.

In [ ]:
@jit(nopython = True)
def calculate_order(h, error):
    #This function calculated the rate of convergence for a method based on step size and error
    n = len(h)
    p = np.zeros(n)
    for i in range(n-1):
        p[i] = np.log(error[i+1]/error[i])/np.log(h[i+1]/h[i])
    p[-1] = p[-2]#Add the last value twice, just to get an array of the same length as h
    #This isn't important because the error for large step size is wrong anyway, we will only consider the values inbetween lower and higher values of step size
    return p

def create_initial_cond(H):
    p2_0 = 0
    q1_0 = 0
    q2_0 = 0.45
    p1_0 = np.sqrt(2/3*q2_0**3 + 2*H_0 - p2_0**2 - q1_0**2 - q2_0**2 - 2*q1_0**2*q2_0)

    return np.array([p1_0, p2_0, q1_0, q2_0])



In [ ]:
H_0 = 1/12 #Initial system energy
y_0 = create_initial_cond(H_0)#Initial conditions for each of the four functions we solve for

t_0 = 0#Start time
t_end = 10#End time 

In [ ]:
a_tol = 1e-17
r_tol = 2.220446049250313e-14#Lowest possible r_tol

number_of_test = 70 #Number of steps between first and last step_size
h_arr = np.logspace(-3.9, 0, number_of_test)#Create an array of various step sizes, that scale logarithmically
SB_error = np.zeros(number_of_test)#Error of Shampine-Bogacki method
kutta_error = np.zeros(number_of_test)#Error of Kutta's method
kahans_error = np.zeros(number_of_test)#Error of Kahan's method
SV_error = np.zeros(number_of_test)#Error of Störmer-Verlet method


t_library_Kutta, y_library_Kutta            =       kuttas_method_library(t_0, t_end, y_0, henon_heiles_library, a_tol, r_tol)   #Library 4th order Runge-Kutta method (reference method)

for i in range(number_of_test):
    t_implemented_SB, y_implemented_SB          =       shampine_bogacki(t_0, t_end, y_0, h_arr[i], henon_heiles)               #Implemented Shampine-Bogacki method
    t_implemented_Kutta, y_implemented_Kutta    =       kuttas_method(t_0, t_end, y_0, h_arr[i], henon_heiles)                  #Implemented Kutta's method
    t_implemented_kahans, y_implemented_kahans  =       kahans_method(t_0, t_end, y_0, h_arr[i], create_A, create_b)            #Implemented Kahan's method
    t_implemented_SV, y_implemented_SV          =       stormer_verlet(t_0, t_end, y_0, h_arr[i], henon_heiles_SV, henon_heiles_SV_last)    #Implmented Störmer-Verlet method

    #Append the errors of each method to the corresponding error-array
    SB_error[i] = np.linalg.norm(y_implemented_SB[:,-1] - y_library_Kutta[:,-1])
    kutta_error[i] = np.linalg.norm(y_implemented_Kutta[:,-1] - y_library_Kutta[:,-1])
    kahans_error[i] = np.linalg.norm(y_implemented_kahans[:,-1] - y_library_Kutta[:,-1])
    SV_error[i] = np.linalg.norm(y_implemented_SV[:,-1] - y_library_Kutta[:,-1])

As

In [ ]:
#Plot error of Bogacki-Shampine method
plt.plot(h_arr, SB_error, c = "r", label = "Shampine-Bogacki method error")
plt.plot(h_arr, SB_error[-1] * h_arr ** 3, ".b", label = "Line with slope 3")
plotCommands("Step size h", "Error", "Rate of convergence test, Shampine-Bogacki method", xscale=True, yscale=True, legend=True)

As we can see from $\bf{\text{Figure 1}}$ the Shampine-bogacki method is correctly implemented as a method of order 3. We can observe that because the (dotted) refernce line with slope 3 is parallel to the error-line. The reason for why the actual error line is a bit above the reference line can be explained by error for large step size, h (here 1), because I am using the last value to determine the constant for the line as explained earlier. 

In [ ]:
#Plot error of Kutta's method
plt.plot(h_arr, kutta_error, c = "r", label = "Kutta's method error")
plt.plot(h_arr, kutta_error[-1] * h_arr ** 4, ".b", label = "Line with slope 4")
plotCommands("Step size h", "Error", "Rate of convergence test Kutta's method", xscale=True, yscale=True, legend=True)

$\bf{\text{Figure 2}}$ shows us rate of convergence for Kutta's method, which is quite obviously 4. We can observe that the line of error is parallel to the line with slop 4. The reason for the inaccuracies for $h < 10^{-3}$ is because of the limitations of the 'RK45' method from the scipy library. As I mentioned earlier, the lowest possible relative tolerance was of order $10^{-14}$, and we can observe that the error analysis only gets inaccurate when the error apporaches said value. 

In [ ]:
#Plot error of Kahan's method
plt.plot(h_arr, kahans_error, c = "r", label = "Kahan's method error")
plt.plot(h_arr, kahans_error[-1] * h_arr ** 2, ".b", label = "Line with slope 2")
plotCommands("Step size h", "Error", "Rate of convergence test Kahan's method", xscale=True, yscale=True, legend=True)

It is very clear from $\bf{\text{Figure 3}}$ that Kahan's method is of order 2, as it is parallel to the line with slope 2.

In [ ]:
#Plot error of Störmer-Verlet method
plt.plot(h_arr, SV_error, c = "r", label = "Störmer-Verlet method error")
plt.plot(h_arr, SV_error[-1] * h_arr ** 2, ".b", label = "Line with slope 2")
plotCommands("Step size h", "Error", "Rate of convergence test Störmer-Verlet method", xscale=True, yscale=True, legend=True)

Because of the same reasons as above, $\bf{\text{Figure 4}}$ shows us that Störmer-Verlet method is of order 2. 

In [ ]:
p_SB = calculate_order(h_arr, SB_error)#Calculate order of Shampine-Bogacki method
p_kutta = calculate_order(h_arr, kutta_error)#Calculate order of Kutta's method
p_kahans = calculate_order(h_arr, kahans_error)#Calculate order of Kahan's method
p_SV = calculate_order(h_arr, SV_error)#Calculate order of Störmer-Verlet method

plt.plot(h_arr, p_SB, label = "Shampine-Bogacki method order")
plt.plot(h_arr, p_kutta, label = "Kutta's method order")
plt.plot(h_arr, p_kahans, label = "Kahan's method order")
plt.plot(h_arr, p_SV, label = "Störmer-Verlet method order")
plotCommands("Step size h", "Rate of convergence p", "Rate of convergence for various numerical methods", xscale = True, legend=True)

Because we have the following relations between the error for two subsequent step sizes, and the order:

$$
E(h_1) \approx C h_1^p, \quad\quad E(h_2)\approx C h_2 ^p
$$

with p as the rate of convergence for the corresponding method, we can combine them and get the following relation:

$$
p \approx \frac{\ln\left(\frac{E(h_2)}{E(h_1)}\right)}{\ln\left(\frac{h_2}{h_1}\right)}
$$

This is what is plotted above in $\bf{\text{Figure 5}}$. The most relevant part for every method is the plot between step sizes $\approx 10^{-3}$ and $\approx 10^{-1}$. For large step sizes this is wrong because, the original assumption that $E(h) \approx C h^p$ assumes $\lim_{h \rightarrow 0}$ [1]. For lower values we have an error because our reference solution (with the scipy library method 'RK45') is not accurate enough. 

As we can see Kutta's method is of order 4, Shampine-Bogacki of order 3, and Kahan's and Störmer-Verlet of order 2.

$\bf{\text{NOTE:}}$ Because the values for large values of h are inaccurate, I have appended the last calculated value for p twice for every method to match the array sizes, and make it easier to plot the results. As explained above, this does not affect the conclusion.

## Problem 2 - Properties of the numerical methods applied on the Hamiltonian of the Hénon-Heiles system


In this section of the report we are going to discuss the properties of each method, by applying them on the Hamiltonian of the Hénon-Heiles system. The Hamiltonian of the Hénon-Heiles system is:

$$
H(q,p) = \frac{1}{2}\left(p_1^2 + p_2^2\right) + U(q)
$$

where $U(q)$ is the potential energy, and is defined as:

$$
U(q) = \frac{1}{2} (q_1^2 + q_2^2) + \lambda(q_1^2 q_2 - \frac{1}{3} q_2^3)
$$

with $\lambda = 1$ in this project.

$\bf{\text{NOTE: }}$ The orange lines are the reference line $H_0$ to show what the constant energy should have been, while the blue graphs show the numerical methods. This applies to all the figures with subplots of each numerical method under this section (problem 2). I couldn't add labels without slowing down the computation time significantly.


In [ ]:
@jit(nopython = True)
def Hamiltonian(Y):#Define the Hamiltonian in our problem
    return 0.5 * (Y[0]**2 + Y[1]**2) + 0.5 * (Y[2]**2 + Y[3]**2) + Y[2]**2 * Y[3] - 1/3 * Y[3]**3

In [ ]:
def calculate_all_methods(t_0, t_end, h, H, showTime = False):#Calculate and return the values for [p1,p2,q1,q2] for all the methods
    y_0 = create_initial_cond(H)

    t_SB, y_SB = shampine_bogacki(t_0, t_end, y_0, h, henon_heiles)
    t_kutta, y_kutta = kuttas_method(t_0, t_end, y_0, h, henon_heiles)
    t_SV, y_SV = stormer_verlet(t_0, t_end, y_0, h, henon_heiles_SV, henon_heiles_SV_last)
    t_kahans, y_kahans = kahans_method(t_0, t_end, y_0, h, create_A, create_b)

    return t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans

def plot_all_methods(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans, N = 10):#Plot all the method's results in one coordinate system
    #We calculate with much smaller time intervals, than we plot, because matplotlib-library is pretty slow
    plt.figure(figsize = (25,10))
    H1 = Hamiltonian(y_SB)[::N]
    H2 = Hamiltonian(y_kutta)[::N]
    H3 = Hamiltonian(y_SV)[::N]
    H4 = Hamiltonian(y_kahans)[::N]

    plt.plot(t_SB[::N], H1, label = "Shampine-Bogacki method")
    plt.plot(t_kutta[::N], H2, label = "Kutta's method")
    plt.plot(t_SV[::N], H3, label = "Störmer-Verlet method")
    plt.plot(t_kahans[::N], H4, label = "Kahan's method")
    plotCommands("Time [s]", "H(q,p)", "Plot of the H(p,q) for various numerical methods", legend = True)

def plot_all_methods_subplot(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans, H_0, N = 10):
    #Plot every method as subplots in the same figure
    #The plots show the deviation of a particular numerical method, from the correct value
    H1 = Hamiltonian(y_SB)[::N]
    H2 = Hamiltonian(y_kutta)[::N]
    H3 = Hamiltonian(y_SV)[::N]
    H4 = Hamiltonian(y_kahans)[::N]

    fig, ax = plt.subplots(2, 2, figsize = (15,15))
    fig.suptitle("Error of Hamiltonian for Hénon-Heiles system for different numerical methods")
    
    ax[0, 0].plot(t_SB[::N], H1, label = "Numerical results")
    ax[0, 0].plot(t_SB[::N], [H_0]*len(t_SB[::N]), label = r"$H_0$ = " + f"{H_0:.3f}")#Reference line for what the energy should be if the method preserved it
    ax[0, 0].set_title("Shampine-Bogacki")

    ax[0, 1].plot(t_kutta[::N], H2, label = "Numerical results")
    ax[0, 1].plot(t_kutta[::N], [H_0]*len(t_kutta[::N]), label = r"$H_0$ = " + f"{H_0:.3f}")
    ax[0, 1].set_title("Kutta")

    ax[1, 0].plot(t_SV[::N], H3, label = "Numerical results")
    ax[1, 0].plot(t_SV[::N], [H_0]*len(t_SV[::N]), label = r"$H_0$ = " + f"{H_0:.3f}")
    ax[1, 0].set_title("Störmer-Verlet")

    ax[1, 1].plot(t_kahans[::N], H4, label = "Numerical results")
    ax[1, 1].plot(t_kahans[::N], [H_0]*len(t_kahans[::N]), label = r"$H_0$ = " + f"{H_0:.3f}")
    ax[1, 1].set_title("Kahan")

    for a in ax.flat:
        a.set(xlabel="Time [s]", ylabel="H(q,p)")

    for i in range(2):
        for j in range(2):
            ax[i, j].ticklabel_format(useOffset=False)
            ax[i, j].grid()
    
    print('\033[1m' + "Figure ", plotCommands.counter)#Bold letters
    plotCommands.counter += 1
    plt.show()

In [ ]:
H_0 = 1/12
t_0 = 0
t_end = 3e5
h = 0.1

t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans = calculate_all_methods(t_0, t_end, h, H_0)
plot_all_methods(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans)
plot_all_methods_subplot(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans, H_0)

From $\bf{\text{Figure 6}}$ above we can observe that neither the Shampine-Bogacki method nor Kutta's method preserves the energy of the system. This is apparent because of the long time interval ($h = 0.1$s (time step), $T = 3 \cdot 10^5$s). While Kahan's method and Störmer-Verlet method oscillates around $H_0$, the other methods drift away from the constant value. 

$\bf{\text{Figure 7}}$ shows the deviation of the numerical results compared to $H_0$. The orange line is $H_0$, and the blue graph is the numerical results in every such plot. From this figure we can observe how much each method deviates. As we discussed earlier, the Hamiltonian oscillates around $H_0$ for Kahan's method and Störmer-Verlet.

In [ ]:
H_0 = 1/12
t_0 = 0
t_end = 3e5
h = 0.01

#I want to time the different numerical methods once, to get an idea of their computation time. This is the choice because of the number of iterations
t = time.time()
t_SB, y_SB = shampine_bogacki(t_0, t_end, y_0, h, henon_heiles)
print("Shampine-Bogacki method time: ", time.time() - t, "s")

t = time.time()
t_kutta, y_kutta = kuttas_method(t_0, t_end, y_0, h, henon_heiles)
print("Kutta's method time: ", time.time() - t, "s")

t = time.time()
t_kahans, y_kahans = kahans_method(t_0, t_end, y_0, h, create_A, create_b)
print("Kahan's method time: ", time.time() - t, "s")

t = time.time()
t_SV, y_SV = stormer_verlet(t_0, t_end, y_0, h, henon_heiles_SV, henon_heiles_SV_last)
print("Störmer-Verlet method time: ", time.time() - t, "s")

plot_all_methods(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans)
plot_all_methods_subplot(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans, H_0)

However from $\bf{\text{Figure 8}}$ we can observe that with a smaller time step $h = 0.01$ (over the same time interval $T = 3\cdot10^5$s), Kutta's method seems to be accurate and is indistinguishable from the calculated results with Kahan's and Störmer-Verlet method. This shows that Kutta's method and the Shampine-Bogacki can be accurate over longer time intervals T, given sufficiently small time steps. The problem is that we can always choose a time interval such that the energy of the systems calculated with the Shampine-Bogacki method and Kutta's method will drift. When we deal with very long time intervals, it most likely will not be practial to use such small time steps. Therefore Störmer-Verlet and Kahan's method are preferable in these situations, even if Shampine-Bogacki and Kutta's method are of higher order. 

In [ ]:
H_0 = 1/12
t_0 = 0
t_end = 3e3
h = 0.01
t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans = calculate_all_methods(t_0, t_end, h, H_0)
plot_all_methods(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans)
plot_all_methods_subplot(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans, H_0)

$\bf{\text{Figure 10}}$ and $\bf{\text{Figure 11}}$ shows the results for $T = 3000$s ($ = 50$ minutes) and $h = 0.01$s. The energy drift as a result of Kutta's method is smaller than the oscillations in Störmer-Verlet and Kahan's method. There are a lot of systems studied in physics where 50 minutes is a lot, and where it is affordable to use smaller time steps to get more accurate results. The Shampine-Bogacki and Kutta's method are suited for such calculations, especially Kutta's method, which only uses a bit more time and computational power than the Shampine-bogacki method, but in return is much more accurate. Both Kahan's method and Störmer-Verlet method are less accurate than the other methods, and is not suited for calculations over such small time intervals. This is also shown in $\bf{\text{Figure 12}}$ and $\bf{\text{Figure 13}}$ for very small time intervals. 

In [ ]:
H_0 = 1/12
t_0 = 0
t_end = 30
h = 0.01
t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans = calculate_all_methods(t_0, t_end, h, H_0)
plot_all_methods_subplot(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans, H_0)

We can observe from $\bf{\text{Figure 12}}$ ($h = 0.01$s, $T = 30$s) that for very small time intervals, even Kutta's method oscillates very heavily. The values it oscillates between, however, are more accurate than the Shampine-Bogacki method.  In addition we can decrease the time step to prevent the oscillations, like shown below in $\bf{\text{Figure 13}}$ ($h = 0.001$s, $T = 30$s).

The only advantages of Shampine-Bogacki method is that it requires fewer function evaluations per step, and is therefore suited for adaptive stepsize calculations. However, for such small time intervals, the computation power required is very small. Above $\bf{\text{Figure 8}}$ the time required to compute each method is noted. We see that for $3 \cdot 10^7$ iterations, the time difference between Shampine-Bogacki method and Kutta's method is not too large. We can also notice that Kahan's method is much slower than the Störmer-Verlet method as it has to solve a system of linear equations at each step, while the Störmer-Verlet method only have 1 function evaluation at each step (except the first and last). Keep in mind that there are sources of error in this, as I for instance could have potentially implemented one or more of the methods more efficiently than others. But the results give us a general idea of the expected computation time. 

In [ ]:
H_0 = 1/12
t_0 = 0
t_end = 30
h = 0.001
t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans = calculate_all_methods(t_0, t_end, h, H_0)
plot_all_methods_subplot(t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans, H_0)

We can observe that even if the range Kahan's method and Störmer-Verlet oscillates between decreases, as we decrease $h$, there still is oscillation present. For short time intervals we want more accurate results, and will more often than not be better off with either Shampine-bogacki or Kutta's method for example. 

### What causes the energy drift in some numerical methods?

We have noticed that none of the four numerical methods used above are energy-preserving. However both the Störmer-Verlet method and Kahan's method seems to preserve some form of the initial energy. We will first study why the Störmer-Verlet method seems to do so.

The Störmer-Verlet method is a symplectic numerical method \[3(p.190)\]. Symplectic methods conserves the Hamiltonian properties of ordinary differential equations \[3 p.187\] (we can consider the solution of a differential equation to be a transformation of the initial conditions \[1\], and symplectic transformations conserves these properties). Symplectic methods does not conserve the energy of the actual Hamiltonian, but rather a perturbed energy of the perturbed Hamiltonian. 

Kahan's method might preserve energy under certain circumstances \[4\]. According to the paper: "When the vector field is Hamiltonian on either a symplectic vectorspace or a Poisson vector space with constant Poisson structure,  the map determined by this discretization has a conserved modified Hamiltonian\[...\]". Because the Hénon-Heiles system seems to preserve some form of the energy (by oscillating around the actual energy, similar to the result calculated by the Störmer-Verlet method), it might be reasonable to conclude that our problem satisfies these conditions. This implies that it is not always appropriate to use Kahan's method when dealing with long time frames. We first have to check if the conditions are met. This method, however, is a implicit method and is therefore generally more stable than the other methods presented in this report. 

Unfortunately I have not been able to find a rigorous source for whether Shampine-Bogacki and Kutta's method are symplectic or not, but according to this wikipedia-article [5](https://en.wikipedia.org/wiki/Symplectic_integrator#Introduction): "Most of the usual numerical methods, like the primitive Euler scheme and the classical Runge–Kutta scheme, are not symplectic integrators.". This coupled with our results, and the fact that both are Runge–Kutta methods, suggests that neither of the methods are symplectic. Otherwise there should have been some form of energy preservation. 

### Conclusion - problem 2

The conclusion is that a numerical method of higher order does not necessarily mean more accurate results. We have to take several factors into consideration when choosing a numerical method for a given problem. In our case we have shown that Shampine-Bogacki method and Kutta's method are quite accurate over relatively small time frames. When dealing with systems that operate over large time intervals, the accuracy at a specific point is less important than the stability of the method over the whole inerval. 

## Problem 3 - Poincaré mapping of the Hénon-Heiles system for different numerical methods

Poncaré-mapping is often used to analyze dynamic, larger dimension, systems in simpler terms. The first main reason for calculating the Poincaré-map of the Hénon-Heiles system, was to determine if there existed a third invariant\[3\].In our case we are going to map all the times $q_1$ in system equals 0, while $p_1 > 0$. 

The method for calculating the Poncaré-map is to iterate through our system and notice when $q_1 = 0$ and $p_1 > 0$. Then we will use interpolation to calculate more accurate values of $q_2$ and $p_2$ which we then will scatter plot in a 2D-coordinate system, thus making it easier to analyze our 4-dimensional system. We will use linear interpolation:

We know that there exists exactly one line through the two points $(t_i, q_{1,i})$ and $(t_{i-1}, q_{1,i-1})$.

$$
P_1(t) = q_{1, i} \frac{t - t_{i-1}}{t_{i} - t_{i-1}} + q_{1, i-1} \frac{t_{} - t_{i-1}}{t_{i} - t_{i-1}}
$$

Here we will define a parameter $\lambda$ with $t = t_i + \lambda h$, and it tells us how much the interpolated point is weighted toward each of the two points we use. For $\lambda = 0$ it is on $(t_{i-1}, q_{1,i-1})$ and on $(t_i, q_{1,i})$ for $\lambda = 1$. And we end up with:

$$
P_1(t) = \lambda q_{1, i} + (1 - \lambda) q_{1, i-1} = 0 \quad\quad (\text{because we want } q_1 = 0)
$$

We will solve for $\lambda$. Because we are dealing with the same time intervals for $q_1, p_1, q_2, p_2$, we can use the same $\lambda$ for every variable. And we solve the equation above for the wanted variable, where the interpolated point is the unknown, and not $\lambda$. However we will only calculate the values of interest, which are $(q_2, p_2)$.



In [ ]:
#Given an array with the same shape as the Y-matricies returned from the numerical methods
#Find the Poincaré-map
#The shape is [[q2_n],[p2_n]]
def poincare(Y):
    size = len(Y[0])
    p_vec = np.zeros((2,1))#Vector containg the intersecting points, [q2_vec, p2_vec], I couldn't get np.empty to work, so I am deleting the first column later
    for i in range(1, size):#We know the intital condition, and therefore know that there is no intersection at the first step, we always choose p_1 to be positive at t_0
        if (Y[:,i][0] > 0 and Y[:,i][2]*Y[:,i-1][2] < 0):#Check if p_1 > 0 and if the sign of q_1 changes
            lam = Y[:,i-1][2]/(Y[:,i-1][2] - Y[:,i][2])#Lambda for interpolation
            p_vec = np.append(p_vec, [[lam * Y[:,i][3] + (1 - lam) * Y[:,i-1][3]], [lam * Y[:,i][1] + (1 - lam) * Y[:,i-1][1]]], axis = 1)#Actual interpolation, as described above

    p_vec = np.delete(p_vec, 0, axis = 1)#Delete the first column with (0,0), because we are using np.zeros()
    return p_vec#Returns all the values of q_2 as the first array and p_2 as second array

def poincare_allmethods(H_0, t_0, t_end_arr, h_arr, colours, labels):
    #Calculate and plot poincare for different time intervals and step sizes for a given H_0
    fig, ax = plt.subplots(2, 2, figsize = (15,15))
    fig.suptitle("Poincaré-map with " + r"$H_0$ = " + f"{H_0:.3f}")
    for i in range(len(h_arr)):
        c = colours[i]
        t_end = t_end_arr[i]
        h = h_arr[i]
        lab = labels[i]

        t_SB, y_SB, t_kutta, y_kutta, t_SV, y_SV, t_kahans, y_kahans = calculate_all_methods(t_0, t_end, h, H_0)
        p_SB = poincare(y_SB)
        p_kutta = poincare(y_kutta)
        p_kahans = poincare(y_kahans)
        p_SV = poincare(y_SV)

        ax[0, 0].scatter(p_SB[0], p_SB[1], color = c, label = lab)
        ax[0, 1].scatter(p_kutta[0], p_kutta[1], color = c, label = lab)
        ax[1, 0].scatter(p_kahans[0], p_kahans[1], color = c, label = lab)
        ax[1, 1].scatter(p_SV[0], p_SV[1], color = c, label = lab)

    ax[0, 0].set_title('Poincaré-map Shampine-Bogacki')
    ax[0, 1].set_title('Poincaré-map Kutta')
    ax[1, 0].set_title('Poincaré-map Kahan')
    ax[1, 1].set_title('Poincaré-map Störmer-Verlet')

    for a in ax.flat:
        a.set(xlabel=r"$q_2$", ylabel=r"$p_2$")

    for i in range(2):
        for j in range(2):
            ax[i, j].ticklabel_format(useOffset=False)
            ax[i, j].grid()
            ax[i, j].legend()
        
    print('\033[1m' + "Figure ", plotCommands.counter)#Bold letters
    plotCommands.counter += 1
    plt.show()

In [ ]:
t_0 = 0
t_end_arr = [1e3, 1e4, 1e5, 1e6]
h_arr = [0.0001, 0.001, 0.01, 0.1]
colours = ["darkorange", "crimson", "lightgreen", "lightcyan"]
labels = ["T = 1e3s & h = 0.0001s", "T = 1e4s & h = 0.001s", "T = 1e5s & h = 0.01s", "T = 1e6s & h = 0.1s"]
H_0 = 1/12
poincare_allmethods(H_0, t_0, t_end_arr, h_arr, colours, labels)

$T$ is the time interval, and $h$ is the time step. We know that the Poincaré-cut for the Hénon-Heiles system with $H_0 = \frac{1}{12}$ should be a closed curve\[1\]. We can observe from $\bf{\text{Figure 14}}$ that Kahan's method and Störmer-Verlet method manages to do this for a time interval of $T = 10^6$s and step size $h = 0.1$s. Kutta's method manages to preserve this property to some degree, but is much better at it when $T = 10^5$s and $h = 0.01$s. It is apparent from the figure that the Shampine-Bogacki method does not fare well with long time intervals and large time steps. The fact that the curve is closed when $T = 10^5$s and $h = 0.01$s, supports our results from problem 2, that Shampine-Bogacki method is only accurate for lower time intervals, unless the step size is extremely small. Because Kutta's method is of higher order, it would take a larger time interval to observe the same, but we should expect the same results at some point. 

## Conclusion

In this project we have discussed the properties of various numerical methods, by studying their application on the Hénon-Heiles system. We have noticed that rate of convergence is not the only property to consider when choosing a numerical method when solving a problem. For problems dealing with shorter time intervals, methods such as Kutta's method or the Shampine-Bogacki method, will yield more accurate results. If we, however, were to study systems in larger time frames, methods such as Störmer-Verlet method and Kahan's method would be preferable. 

## Sources

\[1\] Project description, "TMA4320 - Prosjekt 3 - 2021", March 25, 2021

\[2\] The SciPy community, scipy.integrate.RK45 \[internet\]. \[Last updated 25 March 2021, Accessed 15 April 2021\]. Available from: https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.RK45.html

\[3\] Hairer E, Lubich C, Wanner G. Geometrical Numerical Integration Structur-Preserving Algorithms for Ordinary Differential Equations \[internet\]. Heidelberg: Springer; 2006 \[Accessed 17 April 2021\]. Available from: http://www.mat.unimi.it/users/sansotte/pdf_files/hamsys/HaiLubWan-2006.pdf

\[4\] Celledoni E, McLachlan RI, Owren B. Geometric properties of Kahan's method \[article\]. Trondheim: NTNU; 2012. Available from: https://arxiv.org/pdf/1209.1164.pdf?fbclid=IwAR3MZ12ElEHh9r_1nw8T04-443_KQunVoVKXT6iQAX_Ttu3n1AESewiSArw

\[5\] wikipedia.org. Symplectic integrator. \[Last updated 25 December 2020\]. Available from: https://en.wikipedia.org/wiki/Symplectic_integrator